In [13]:
#Importar galerias
import numpy as np
import pandas as pd
import random
from scipy.stats import t

In [14]:
#PREGUNTA 3 #########################################################################
#Y, Proceso generador de datos con 5 variables (1 + x1 + x2 + x3 + x4 )

np.random.seed(170)

x1 = np.random.rand(10000)  
x2 = np.random.rand(10000)  
x3 = np.random.rand(10000)  
x4 = np.random.rand(10000) 
x5 = np.random.rand(10000)  
e = np.random.normal(0,1,10000) # normal distribution


Y = 1 + 0.8*x1 + 1.2*x2 + 0.5*x3 + 1.5*x4  + e
print(Y.size)  #10 mil observaciones

10000


In [15]:
#Informacion omitiendo una variable, se omite x4
mat = np.column_stack((Y, np.ones(10000),x1,x2,x3)) 

In [16]:
muestra = (10, 50, 80 , 120 , 200, 500, 800, 1000, 5000)

for m in muestra:
    
    elegidos=random.sample(range(10000), m)     #vector de indices de las posiciones de las observaciones elegidos en la muestra
    observ= mat[elegidos,:]                     #matriz con observaciones elegidas
   
    #vector de X
    X = observ[:,1:] 
    #vector de Y
    Y = observ[:,0]                             
    
    #1. Construir beta
    beta = np.linalg.inv(X.T @ X) @ ((X.T) @ Y )
    beta

    #2. Error estandar
    y_est =  X @ beta                                                        #2.b. Y estimado = vector 
        
    n  = X.shape[0]                                                          #2.c. numero de observaciones
        
    k  = X.shape[1] - 1                                                      #2.d. x - unos
        
    nk = n - k                                                               #2.e. grados de libertad
        
    sigma2 =  sum(list( map( lambda x: x**2 , Y - y_est)   )) / nk           #2.f. SCR/ (n-k) = e'e /(n-k) = s^2 =sigma^2
        
    Var   = sigma2*np.linalg.inv(X.T @ X)                                     #2.g. Var_Cov(betas) = s^2 * (X'X)-1
    
    sd    = np.sqrt( np.diag(Var) )                                          #2.h. desv(betas)=var(betas)^(1/2)

    #3. Tamano de la muestra m
    tamano=np.ones(len(beta))*m                                              # se crea vector para colocarlo en la tabla
    
    
    df    = pd.DataFrame( {"Muestra_tamano": tamano, "OLS": beta , "standar_error" : sd })  
    print(df)

   Muestra_tamano       OLS  standar_error
0            10.0  1.762864       1.034679
1            10.0 -1.289283       1.183143
2            10.0  1.703140       1.131276
3            10.0  2.565389       1.341874
   Muestra_tamano       OLS  standar_error
0            50.0  2.330939       0.564480
1            50.0  0.226899       0.557084
2            50.0  0.890242       0.557368
3            50.0  0.456047       0.535727
   Muestra_tamano       OLS  standar_error
0            80.0  1.848696       0.386030
1            80.0  0.614000       0.398003
2            80.0  1.717269       0.399618
3            80.0  0.173237       0.379192
   Muestra_tamano       OLS  standar_error
0           120.0  1.239140       0.292674
1           120.0  0.806612       0.310058
2           120.0  1.370872       0.321418
3           120.0  1.079093       0.334514
   Muestra_tamano       OLS  standar_error
0           200.0  1.562910       0.239431
1           200.0  0.929787       0.265135
2          

In [17]:
#PREGUNTA 4 ########################################################################
# Y, proceso generador de datos con 8 variables 

np.random.seed(170)

x1 = np.random.rand(800)  
x2 = np.random.rand(800)  
x3 = np.random.rand(800)  
x4 = np.random.rand(800) 
x5 = np.random.rand(800)  
x6 = np.random.rand(800) 
x7 = np.random.rand(800) 
e = np.random.normal(0,1,800) # normal distribution

Y = 1 + 0.8*x1 + 1.2*x2 + 0.5*x3 + 1.5*x4 + 2*x5 + 3*x6 + 1.5*x7 + e
print(Y.size)   #800 observaciones

X = np.column_stack((np.ones(800),x1,x2,x3,x4,x5,x6,x7))

800


In [18]:
#1. Definir funcion          
    #Incluye,
        #argumento de que incluye intercepto por default
        #argumento de heterocedasticidad     (es homocedastico por default)
        
def ols(M,Y, inter=True, homo=True):
    
#2. Estimaciones
    #Beta
    beta = np.linalg.inv(X.T @ X) @ ((X.T) @ Y )                             #2.a. estimar beta
        
    y_est =  X @ beta                                                        #2.b. Y estimado  y Y media
    y_mean=np.mean(Y) 
    
    n  = X.shape[0]                                                          #2.c. numero de observaciones
    k  = X.shape[1] - 1                                                      #2.d. numeros de x - unos 
    nk = n - k                                                               #2.e. grados de libertad
    
    ee= list( map( lambda x: x**2 , Y - y_est))                              #2.f. Vector de residuos^2
        
    SCR =  sum(ee)                                                           #2.g. Sumatoria Cuadrado de Residuos
    SCT =  sum(list( map( lambda x: x**2 , Y - y_mean)))                     #2.h. Sumatoria Cuadrados Totales
    
    # Root-MSE    
    root_mse = np.sqrt(SCR/ n)                                          
    
    # R cuadrado
    R_cuadrado = 1- SCR/SCT
    
    diccionario   = {'R2': R_cuadrado ,'Rootmse': root_mse  }
        
    if inter and homo: 
        
        sigma2 =  SCR / nk                                                    #2.f. SCR/ (n-k) = e'e /(n-k) = s^2
        
        Var   = sigma2*np.linalg.inv(X.T @ X)                                 #2.g. Var_Cov(betas) = s^2 * (X'X)-1
      
        sd    = np.sqrt( np.diag(Var) )                                       #2.h. desv(betas)=var(betas)^1/2 , solo diag
                                                                                        
        t_est = np.absolute(beta/sd)                                          #2.i. t_est = |beta-0|/sd ==> para cada beta 
        
        pvalue= (1 - t.cdf(t_est, df=nk) ) * 2                                #2.j. p_value ==> para cada beta
     
        superior= beta + 1.96 * sd                                            #2.h. límites 
        inferior= beta - 1.96 * sd
        
        df = pd.DataFrame( {"Estimacion_OLS": beta , "standar_error" : sd , "Pvalue" : pvalue, "Limit_sup": superior, "Limit_inf": inferior}) 
      
    elif inter and (homo is False):   

        White= np.eye(n)*ee                                                      #2.a. Matriz de corrección de White
        
        VarCorg=np.linalg.inv(X.T @ X) @ X.T @White @ X @ np.linalg.inv(X.T@X)   #2.g. Var_Cov(betas) corregida             
        
        sd    = np.sqrt( np.diag(VarCorg) )                                      #2.h. desv(betas)=var(betas)^1/2 , solo diag
                                                                                        
        t_est = np.absolute(beta/sd)                                             #2.i. t_est =|beta-0|/sd ==> para cada beta 
        
        pvalue= (1 - t.cdf(t_est, df=nk) ) * 2                                   #2.j. p_value ==> para cada beta
     
        superior= beta + 1.96 * sd                                               #2.h limites
        inferior= beta - 1.96 * sd
        
        df = pd.DataFrame( {"Estimacion_OLS": beta , "standar_error" : sd , "Pvalue" : pvalue, "Limit_sup": superior, "Limit_inf": inferior}) 
      
    print(df, "\n", diccionario)

In [19]:
ols(X,Y)

   Estimacion_OLS  standar_error        Pvalue  Limit_sup  Limit_inf
0        1.214653       0.165610  5.508927e-13   1.539248   0.890058
1        0.728137       0.123197  5.067673e-09   0.969603   0.486671
2        1.247746       0.120894  0.000000e+00   1.484699   1.010793
3        0.532272       0.122616  1.602150e-05   0.772599   0.291945
4        1.239678       0.120271  0.000000e+00   1.475409   1.003946
5        1.829955       0.121558  0.000000e+00   2.068208   1.591702
6        2.992847       0.120800  0.000000e+00   3.229616   2.756079
7        1.477507       0.123787  0.000000e+00   1.720129   1.234886 
 {'R2': 0.6090321142428282, 'Rootmse': 0.9918590768405482}


In [ ]:
ols(X,Y,homo=False)             
#Cuando hay heterocedasticidad, la varianza modificada afecta el error estandar, pvalu, limites.
#No cambia los R2 y Rootmse pues solo explican como los betas estimados ajustan la linea de regresion, y con la matriz de white
#los betas no han cambiado